### Load already trained weights but improve concat2 weights by a good init.

In [1]:
import caffe
import numpy as np

In [25]:
fuse_deploy_concat012 = '/home/jguerry/workspace/dl_data_sunrgbd/MODELS/20170301_fcn32s_optim_fuse_concat/deploy_fuse_concat012.prototxt'
weights_path_fuse_concat012='/home/jguerry/workspace/dl_data_sunrgbd/MODELS/20170301_fcn32s_optim_fuse_concat/astroboy_snapshot_iter_36995.caffemodel'
weights_path_fuse_concat012_output = '/home/jguerry/workspace/dl_data_sunrgbd/MODELS/20170301_fcn32s_optim_fuse_concat/fuse_concat012_retouched.caffemodel'

In [3]:
caffe_gpu = True
if not caffe_gpu:
    caffe.set_mode_cpu()
else:
    caffe.set_mode_gpu()
    caffe.set_device(0);

In [4]:
fuseNet_concat = caffe.Net(fuse_deploy_concat012, weights_path_fuse_concat012, caffe.TEST)

In [8]:
fuseBlobs = [(k, v.data.shape) for k, v in fuseNet_concat.blobs.items()]
fuseLayers = [(k, v[0].data.shape, v[1].data.shape) for k, v in fuseNet_concat.params.items() if 'upscore' not in k]

In [11]:
fuseBlobs

[('data0', (1, 3, 200, 200)),
 ('data1', (1, 3, 200, 200)),
 ('data0_data', (1, 3, 530, 730)),
 ('data0_data_data0_data_0_split_0', (1, 3, 530, 730)),
 ('data0_data_data0_data_0_split_1', (1, 3, 530, 730)),
 ('data1_data', (1, 3, 530, 730)),
 ('data1_data_data1_data_0_split_0', (1, 3, 530, 730)),
 ('data1_data_data1_data_0_split_1', (1, 3, 530, 730)),
 ('label', (1, 1, 530, 730)),
 ('label_label_0_split_0', (1, 1, 530, 730)),
 ('label_label_0_split_1', (1, 1, 530, 730)),
 ('label_label_0_split_2', (1, 1, 530, 730)),
 ('data0_conv1_1', (1, 64, 530, 730)),
 ('data0_conv1_2', (1, 64, 530, 730)),
 ('data0_pool1', (1, 64, 265, 365)),
 ('data0_conv2_1', (1, 128, 265, 365)),
 ('data0_conv2_2', (1, 128, 265, 365)),
 ('data0_pool2', (1, 128, 133, 183)),
 ('data0_conv3_1', (1, 256, 133, 183)),
 ('data0_conv3_2', (1, 256, 133, 183)),
 ('data0_conv3_3', (1, 256, 133, 183)),
 ('data0_pool3', (1, 256, 67, 92)),
 ('data0_conv4_1', (1, 512, 67, 92)),
 ('data0_conv4_2', (1, 512, 67, 92)),
 ('data0_conv

In [10]:
fuseLayers

[('data0_conv1_1', (64, 3, 3, 3), (64,)),
 ('data0_conv1_2', (64, 64, 3, 3), (64,)),
 ('data0_conv2_1', (128, 64, 3, 3), (128,)),
 ('data0_conv2_2', (128, 128, 3, 3), (128,)),
 ('data0_conv3_1', (256, 128, 3, 3), (256,)),
 ('data0_conv3_2', (256, 256, 3, 3), (256,)),
 ('data0_conv3_3', (256, 256, 3, 3), (256,)),
 ('data0_conv4_1', (512, 256, 3, 3), (512,)),
 ('data0_conv4_2', (512, 512, 3, 3), (512,)),
 ('data0_conv4_3', (512, 512, 3, 3), (512,)),
 ('data0_conv5_1', (512, 512, 3, 3), (512,)),
 ('data0_conv5_2', (512, 512, 3, 3), (512,)),
 ('data0_conv5_3', (512, 512, 3, 3), (512,)),
 ('data0_fc6', (4096, 512, 7, 7), (4096,)),
 ('data0_fc7', (4096, 4096, 1, 1), (4096,)),
 ('data0_score_fr37', (37, 4096, 1, 1), (37,)),
 ('data1_conv1_1', (64, 3, 3, 3), (64,)),
 ('data1_conv1_2', (64, 64, 3, 3), (64,)),
 ('data1_conv2_1', (128, 64, 3, 3), (128,)),
 ('data1_conv2_2', (128, 128, 3, 3), (128,)),
 ('data1_conv3_1', (256, 128, 3, 3), (256,)),
 ('data1_conv3_2', (256, 256, 3, 3), (256,)),
 ('da

In [24]:
fuseNet_concat.params['concat2_fc7'][0].data[:,:4096,...]=fuseNet_concat.params['data0_fc7'][0].data[:,:4096,...]
fuseNet_concat.params['concat2_fc7'][0].data[:,4096:,...]=fuseNet_concat.params['data1_fc7'][0].data[:,:4096,...]
fuseNet_concat.params['concat2_fc7'][1].data[...]= 0.5*fuseNet_concat.params['data0_fc7'][1].data[...]+0.5*fuseNet_concat.params['data1_fc7'][1].data[...]

In [26]:
#### SAVING
print "Saving output model to %s " % weights_path_fuse_concat012_output
fuseNet_concat.save(weights_path_fuse_concat012_output)

Saving output model to /home/jguerry/workspace/dl_data_sunrgbd/MODELS/20170301_fcn32s_optim_fuse_concat/fuse_concat012_retouched.caffemodel 


In [27]:
del fuseNet_concat